
Copyright 2024 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


# Batching

In [2]:
import transformer_lens, torch, tqdm, copy, itertools
import numpy as np
from pathlib import Path
from toolz import compose

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter, MaxNLocator

mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False

import sys
sys.path.append('/home/jupyter/')
from paraMem.utils import modelHandlers, dataLoaders

In [4]:
(mem_prompts,mem_counts),(non_mem_prompts,non_mem_counts) = dataLoaders.load_pile_splits("mock_em")
mem_dl = torch.utils.data.DataLoader(mem_prompts, batch_size=1, shuffle=True)
non_mem_dl = torch.utils.data.DataLoader(non_mem_prompts, batch_size=4, shuffle=True)

In [52]:
c_n, k_n = 10, 50
c_idcs, k_idcs = torch.arange(0, c_n, 1), torch.arange(0, k_n, 1)
#c_idcs, k_idcs = torch.randperm(c_n), torch.randperm(k_n)

c_idcs = torch.tensor_split(c_idcs, 2, dim=0)
k_idcs = torch.tensor_split(k_idcs, 5, dim=0)

print(c_idcs)

#c_idcs = torch.tensor(list(map(lambda idcs: (idcs[0].item(), idcs[-1].item()), torch.tensor_split(c_idcs, 2, dim=0))))
#k_idcs = torch.tensor(list(map(lambda idcs: (idcs[0].item(), idcs[-1].item()), torch.tensor_split(k_idcs, 5, dim=0))))

(tensor([0, 1, 2, 3, 4]), tensor([5, 6, 7, 8, 9]))


In [72]:
class ContrastDataset(torch.utils.data.Dataset):
    def __init__(self, c_set, k_set, c_batch:int=5, k_batch:int=10, shuffle:bool=True):
        super().__init__()
        self.c_set, self.c_batch, self.c_n = torch.LongTensor(c_set), c_batch, c_set.shape[0]
        self.k_set, self.k_batch, self.k_n = torch.LongTensor(k_set), k_batch, k_set.shape[0]
        self.c_n_batches, self.k_n_batches = int(self.c_n / self.c_batch), int(self.k_n / self.k_batch)
        self.shuffle = shuffle
        
        ## prepare batched dataset__________________________
        self.c_idcs, self.k_idcs = torch.arange(0, self.c_n, 1), torch.arange(0, self.k_n, 1)
        self.c_idcs_batched = torch.tensor_split(self.c_idcs, self.c_n_batches, dim=0)
        self.k_idcs_batched = torch.tensor_split(self.k_idcs, self.k_n_batches, dim=0)
        
    def reshuffle(self,):
        c_idcs, k_idcs = torch.randperm(self.c_n), torch.randperm(self.k_n)
        self.c_idcs_batched = torch.tensor_split(c_idcs, self.c_n_batches, dim=0)
        self.k_idcs_batched = torch.tensor_split(k_idcs, self.k_n_batches, dim=0)        
        
    def __getitem__(self, idx):
        if self.shuffle:
            self.reshuffle()  
        c_set = self.c_set[self.c_idcs_batched[idx].flatten(),:]
        k_set = self.k_set[self.k_idcs_batched[idx].flatten(),:]
        return c_set, k_set

    def __len__(self): ## choose the shorter set
        if self.c_n_batches < self.k_n_batches:
            return self.c_n_batches
        else:
            return self.k_n_batches
                
ds = ContrastDataset(mem_prompts, non_mem_prompts, c_batch=4, k_batch=10, shuffle=False)
dl = torch.utils.data.DataLoader(ds, batch_size=1, shuffle=False)

for (c, k) in dl:
    print(c.shape, k.shape)

torch.Size([1, 4, 100]) torch.Size([1, 11, 100])
torch.Size([1, 4, 100]) torch.Size([1, 11, 100])
torch.Size([1, 4, 100]) torch.Size([1, 10, 100])
torch.Size([1, 4, 100]) torch.Size([1, 10, 100])
torch.Size([1, 4, 100]) torch.Size([1, 10, 100])
torch.Size([1, 4, 100]) torch.Size([1, 10, 100])
torch.Size([1, 4, 100]) torch.Size([1, 10, 100])
torch.Size([1, 4, 100]) torch.Size([1, 10, 100])
torch.Size([1, 4, 100]) torch.Size([1, 10, 100])
torch.Size([1, 4, 100]) torch.Size([1, 10, 100])
torch.Size([1, 4, 100]) torch.Size([1, 10, 100])
torch.Size([1, 4, 100]) torch.Size([1, 10, 100])
torch.Size([1, 4, 100]) torch.Size([1, 10, 100])
torch.Size([1, 4, 100]) torch.Size([1, 10, 100])
torch.Size([1, 4, 100]) torch.Size([1, 10, 100])
torch.Size([1, 4, 100]) torch.Size([1, 10, 100])
torch.Size([1, 4, 100]) torch.Size([1, 10, 100])
torch.Size([1, 4, 100]) torch.Size([1, 10, 100])
torch.Size([1, 4, 100]) torch.Size([1, 10, 100])
torch.Size([1, 4, 100]) torch.Size([1, 10, 100])
torch.Size([1, 4, 10

In [35]:
for data in zip(mem_dl, non_mem_dl):
    print(len(data), data[0].shape)

2 torch.Size([1, 100])
2 torch.Size([1, 100])
2 torch.Size([1, 100])
2 torch.Size([1, 100])
2 torch.Size([1, 100])
2 torch.Size([1, 100])
2 torch.Size([1, 100])
2 torch.Size([1, 100])
2 torch.Size([1, 100])
2 torch.Size([1, 100])
2 torch.Size([1, 100])
2 torch.Size([1, 100])
2 torch.Size([1, 100])
2 torch.Size([1, 100])
2 torch.Size([1, 100])
2 torch.Size([1, 100])
2 torch.Size([1, 100])
2 torch.Size([1, 100])
2 torch.Size([1, 100])
2 torch.Size([1, 100])
2 torch.Size([1, 100])
2 torch.Size([1, 100])
2 torch.Size([1, 100])
2 torch.Size([1, 100])
2 torch.Size([1, 100])
2 torch.Size([1, 100])
2 torch.Size([1, 100])
2 torch.Size([1, 100])
2 torch.Size([1, 100])
2 torch.Size([1, 100])
2 torch.Size([1, 100])
2 torch.Size([1, 100])
2 torch.Size([1, 100])
2 torch.Size([1, 100])
2 torch.Size([1, 100])
2 torch.Size([1, 100])


In [22]:
def batched_pile(mem_batch:int=1, non_mem_batch:int=5, load_data:str="mock_em", test_frac:float=0.0, shuffle:bool=True):
    (mem_prompts,mem_counts),(non_mem_prompts,non_mem_counts) = dataLoaders.load_pile_splits(load_data, as_torch=True)\
    
    n_mem, n_non_mem = mem_prompts.shape[0], non_mem_prompts.shape[0]
    n_test_mem, n_test_non_mem = int(test_frac*n_mem), int(test_frac*n_non_mem)
    train_mem, test_mem = torch.utils.data.random_split(mem_prompts, [n_mem-n_test_mem, n_test_mem])
    train_non_mem, test_non_mem = torch.utils.data.random_split(non_mem_prompts, [n_non_mem-n_test_non_mem, n_test_non_mem])

    train_mem_dl = torch.utils.data.DataLoader(train_mem, batch_size=mem_batch, shuffle=shuffle)
    train_non_mem_dl = torch.utils.data.DataLoader(train_non_mem, batch_size=non_mem_batch, shuffle=shuffle)
    
    test_mem_dl, test_non_mem_dl = iter([]), iter([])
    if test_frac > 0: ## empty iterable
        test_mem_dl = torch.utils.data.DataLoader(test_mem, batch_size=mem_batch, shuffle=shuffle)
        test_non_mem_dl = torch.utils.data.DataLoader(test_non_mem, batch_size=non_mem_batch, shuffle=shuffle)

    train_dl = zip(itertools.cycle(train_mem_dl), train_non_mem_dl)
    test_dl = zip(itertools.cycle(test_mem_dl), test_non_mem_dl)
    return train_dl, test_dl

train_dl, test_dl = batched_pile(mem_batch=10, non_mem_batch=20, test_frac=0.0)

In [16]:
train_dl